# Lab 09-2

## 5 steps of using TensorBoard

1. From TF graph, decide which tensors you want to log
    ```python
    w2_hist = tf.summary.histogram("weights2", W2) # multi-dimensional tensors
    cost_summ = tf.summary.scalar("cost", cost)
    ```

2. Merge all summaries
    ```python
    summary = tf.summary.merge_all()
    ```

3. Create writer and add graph
    ```python
    writer = tf.summary.FileWriter('./logs')
    writer.add_graph(sess.graph)
    ```

4. Run summary merge and add summary
    ```python
    s, _ = sess.run([summary, optimizer], feed_dict=feed_dict)
    writer.add_summary(s, global_step=global_step)
    ```

5. Launch TensorBoard
    ```python
    tensorboard --logdir=./logs
    ```

## Add scope for better graph hierarchy

```python
with tf.name_scope('layer1') as scope:
    ...
    
with tf.name_scope('layer2') as scope:
    ...
```

## Multiple runs

```tf.summary.FileWriter```에 ```logs/xor_logs, logs/xor_logs_r0_01```과 같이 하위 디렉토리 이름을 다르게 해서 저장한 후, ```tensorboard --logdir=./logs```를 이용해 보면 된다.

## Exercise - TensorBoard for MNIST

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

W1 = tf.Variable(tf.random_normal([784, 40]), name='weight1')
b1 = tf.Variable(tf.random_normal([40]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([40, 40]), name='weight2')
b2 = tf.Variable(tf.random_normal([40]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([40, 40]), name='weight3')
b3 = tf.Variable(tf.random_normal([40]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([40, 40]), name='weight4')
b4 = tf.Variable(tf.random_normal([40]), name='bias4')
layer4 = tf.sigmoid(tf.matmul(layer3, W4) + b4)

W5 = tf.Variable(tf.random_normal([40, 10]), name='weight5')
b5 = tf.Variable(tf.random_normal([10]), name='bias5')
hypothesis = tf.nn.softmax(tf.matmul(layer4, W5) + b5)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train = optimizer.minimize(cost)

prediction = tf.argmax(hypothesis, axis=1)
is_correct = tf.equal(prediction, tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

epochs = 15
batch_size = 100
num_iteartions = int(mnist.train.images.shape[0] / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

tf.summary.histogram('weight1', W1)
tf.summary.histogram('weight2', W2)
tf.summary.histogram('weight3', W3)
tf.summary.histogram('weight4', W4)
tf.summary.histogram('weight5', W5)
tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

summary = tf.summary.merge_all()

writer = tf.summary.FileWriter('./logs')
writer.add_graph(sess.graph)

for epoch in range(epochs):
    total_cost = 0

    for i in range(num_iteartions):
        batch_x_data, batch_y_data = mnist.train.next_batch(batch_size)

        s, cost_val, _ = sess.run([summary, cost, train], feed_dict={X: batch_x_data, Y: batch_y_data})

        total_cost += cost_val
        writer.add_summary(s, global_step=(epoch * num_iteartions + i))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(total_cost / num_iteartions))

acc_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Accuracy: ", acc_val)

/home/jinhyuk/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost = 1.776198500
Epoch: 0002 cost = 1.103110467
Epoch: 0003 cost = 0.856227727
Epoch: 0004 cost = 0.719479916
Epoch: 0005 cost = 0.630894968
Epoch: 0006 cost = 0.569035370
Epoch: 0007 cost = 0.523353374
Epoch: 0008 cost = 0.488216158
Epoch: 0009 cost = 0.459423770
Epoch: 0010 cost = 0.435303340
Epoch: 0011 cost = 0.414649513
Epoch: 0012 cost = 0.397563543
Epoch: 0013 cost = 0.381769914
Epoch: 0014 cost = 0.368472253
Epoch: 0015 cost = 0.356472811
Accuracy:  0.8867
